In [1]:
import sys
import numpy as np

sys.path.append('../')

import cupy
from mandala import cuda
from mandala import Node
from mandala import Variable

from mandala.autodiff import autodiff
from mandala.autodiff import layer
from mandala.autodiff import initializers

In [2]:
x = Variable(np.zeros((8, 3, 10, 10), dtype=np.float32))

In [2]:
if cuda.cudnn_enable:
    cudnn = cuda.cudnn

In [55]:
    def _get_out_size(self, inputs):
        x, W = inputs[:2]
        _, _, kh, kw = W.shape
        _, _, h, w = x.shape
        out_h = conv.get_conv_outsize(
            h, kh, self.sy, self.ph, cover_all=self.cover_all, d=self.dy)
        if out_h <= 0:
            raise RuntimeError('Height in the output should be positive.')
        out_w = conv.get_conv_outsize(
            w, kw, self.sx, self.pw, cover_all=self.cover_all, d=self.dx)
        if out_w <= 0:
            raise RuntimeError('Width in the output should be positive.')
        return out_h, out_w

In [4]:
import chainer.links as L

In [5]:
test = L.Convolution2D(10, 20, 3)

In [6]:
test.W.shape

(20, 10, 3, 3)

In [13]:
x = np.zeros((8, 1, 1, 3), dtype=np.float32)
W = np.zeros((5, 3), dtype=np.float32)
np.dot(x, W.T).shape

(8, 1, 1, 5)

In [40]:
x = cupy.arange(150, dtype=np.float32).reshape(2, 3, 5, 5)

In [44]:
x = cupy.zeros((8, 1024, 10, 10), dtype=np.float32)

In [49]:
ind = np.random.randint(0, 3, 8 * 1024 * 10 * 10)

In [55]:
y = x[ind, ind, ind, ind]#.reshape(8, 1024, 10, 10)

In [65]:
get_ind_size(5, 3, 2)

2

In [57]:
import itertools

In [ ]:
ind_b_base = np.arange(0, 8)
ind_h_base

In [206]:
def make_indices(b, c, h, w, kh, kw, sy, sx, dy, dx):
    h_ind = get_ind_size(h, kh, sy, dy)
    w_ind = get_ind_size(w, kw, sx, dx)
    dim_col = b * c * kh * kh
    indices = np.empty((h_ind, w_ind, dim_col, 4), dtype=np.int32)
    
    ind_b_base = np.arange(0, b, dtype=np.int32)
    ind_c_base = np.arange(0, c, dtype=np.int32)
    ind_h_base = np.arange(0, kh, dtype=np.int32) * dy
    ind_w_base = np.arange(0, kw, dtype=np.int32) * dx

    ind_base = np.array(
        list(itertools.product(ind_b_base,
                               ind_c_base,
                               ind_h_base,
                               ind_w_base)), dtype=np.int32)
    indices[:, :] = ind_base

    grid_b, grid_c, grid_h, grid_w = np.meshgrid(
        ind_b_base,
        ind_c_base,
        np.arange(0, h_ind, sy),
        np.arange(0, w_ind, sx))
    indices[:, :, :, 0] += grid_b
    return indices

In [3]:
xp = cupy

In [4]:
def get_ind_size(size, k, s, d=1):
    dk = k + (k - 1) * (d - 1)
    return (size - dk) // s + 1

In [5]:
def make_indices(b, c, h, w, kh, kw, sy, sx, dy, dx):
    h_ind = get_ind_size(h, kh, sy, dy)
    w_ind = get_ind_size(w, kw, sx, dx)
    indices = xp.empty((b, c, kh, kw, h_ind, w_ind, 4), dtype=xp.int32)

    ind_b_base = xp.arange(0, b, dtype=np.int32)
    ind_c_base = xp.arange(0, c, dtype=np.int32)
    ind_h_base = xp.arange(0, kh, dtype=np.int32) * dy
    ind_w_base = xp.arange(0, kw, dtype=np.int32) * dx
    
    indices[..., 0] = ind_b_base[:, None, None, None, None, None]
    indices[..., 1] = ind_c_base[None, :, None, None, None, None]
    indices[..., 2] = ind_h_base[None, None, :, None, None, None]
    indices[..., 3] = ind_w_base[None, None, None, :, None, None]
    
    grid_x, grid_y = xp.meshgrid(
        xp.arange(w_ind, dtype=np.int32) * sx,
        xp.arange(h_ind, dtype=np.int32) * sy)
    
    indices[..., 2] += grid_y
    indices[..., 3] += grid_x

    return indices

In [6]:
b = 32
c = 1024
h = 7
w = 7
x = xp.arange(b * c * h * w).reshape(b, c, h, w)

In [7]:
ind = make_indices(b, c, h, w, 3, 3, 1, 1, 1, 1)

In [8]:
%%timeit
#ind = make_indices(b, c, h, w, 3, 3, 1, 1, 1, 1)
ind_b = ind[..., 0].ravel()
ind_c = ind[..., 1].ravel()
ind_h = ind[..., 2].ravel()
ind_w = ind[..., 3].ravel()
x[ind_b, ind_c, ind_h, ind_w].get()

35.7 ms ± 221 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
from chainer.utils.conv import im2col_cpu, im2col_gpu

In [19]:
%%timeit
im2col(x, 3, 3, 1, 1, 0, 0).get()

23.9 ms ± 56.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
from mandala import cuda


def get_conv_outsize(size, k, s, p, cover_all=False, d=1):
    dk = k + (k - 1) * (d - 1)
    if cover_all:
        return (size + p * 2 - dk + s - 1) // s + 1
    else:
        return (size + p * 2 - dk) // s + 1


def im2col(img, kh, kw, sy, sx, ph, pw, pval=0, cover_all=False,
           dy=1, dx=1, out_h=None, out_w=None):
    xp = cuda.get_array_module(img)
    n, c, h, w = img.shape
    if out_h is None:
        out_h = get_conv_outsize(h, kh, sy, ph, cover_all, dy)
    if out_w is None:
        out_w = get_conv_outsize(w, kw, sx, pw, cover_all, dx)

    img = xp.pad(img,
                 ((0, 0), (0, 0), (ph, ph + sy - 1), (pw, pw + sx - 1)),
                  mode='constant', constant_values=(pval,))
    col = xp.ndarray((n, c, kh, kw, out_h, out_w), dtype=img.dtype)

    for j in range(kh):
        jdy = j * dy
        j_lim = jdy + sy * out_h
        for i in range(kw):
            idx = i * dx
            i_lim = idx + sx * out_w
            col[:, :, j, i, :, :] = img[:, :, jdy:j_lim:sy, idx:i_lim:sx]

    return col

In [21]:
col = im2col(x, 3, 3, 1, 1, 0, 0)

In [30]:
W = xp.ones((10, 1024, 3, 3), dtype=np.float32)

In [36]:
xp.tensordot(col, W, ((1, 2, 3), (1, 2, 3))).shape

(32, 5, 5, 10)

In [ ]:
def forward_convolution_2d(x, W, b, stride, pad,
                           cover_all=None, dilate=(1, 1)):
    _, _, kh, kw = W.shape
    sy, sx = stride
    ph, pw = pad
    dy, dx = dilate

    col = im2col(x, kh, kw, sy, )